# Projeto 1 - Ciência dos Dados

Nome: Pedro De Lucca Sartori Chagas Ferro

Nome: Pedro Antônio Braga Dutra

___
Carregando algumas bibliotecas:

In [54]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [55]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Pedro De Lucca\Documents\Insper\2° Semestre\CDados\22-2a-cd-p1-grupo_pedrolscf\notebooks


In [56]:
filename = 'dados.xlsx'

In [57]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target,P_Chave
0,bolsonaro,"Após repercussão, site com críticas ao preside...","Após a repercussão, o domínio ‘bolsonaro.com.b...",01/09/2022 15:50,1,0,crítica e acusações sobre a família bolsonaro
1,bolsonaro,Bolsonaro chama de narrativa críticas de compo...,O presidente Jair Bolsonaro (PL) atribuiu a um...,01/09/2022 20:56,1,0,ataques à jornalista
2,bolsonaro,"Lula cai dois pontos, Bolsonaro fica estável, ...",Uma nova pesquisa Datafolha presidencial foi d...,01/09/2022 21:42,1,0,NaN
3,bolsonaro,"Datafolha: Entre mulheres, Lula tem 48%, Bolso...",A pesquisa do Instituto Datafolha divulgada ne...,01/09/2022 22:01,1,0,protagonizou agressão verbal à jornalista vera
4,bolsonaro,Lula diz que orçamento de Bolsonaro não prevê ...,O ex-presidente e candidato Luiz Inácio Lula d...,01/09/2022 22:24,1,0,não inclui reajuste do salário mínimo


In [58]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,Descrição,Data,Pagina,Target,P_Chave
0,bolsonaro,Bolsonaro critica TSE e diz que medidas da Cor...,O presidente Jair Bolsonaro (PL) fez críticas ...,02/09/2022 15:30,1,0,críticas TSE
1,bolsonaro,"Esquerda se solidarizou com facada em 2018, ao...",Diferentemente do que disse o presidente Jair ...,02/09/2022 17:48,1,0,contrário diferentemente Bolsonaro
2,bolsonaro,Bolsonaro cometeu crime contra a humanidade na...,O júri simbólico do Tribunal Permanente dos Po...,01/09/2022 14:08,2,0,condenou crime sentença Bolsonaro
3,bolsonaro,Crítico de Bolsonaro compra domínio e ‘recria’...,Um site que antes era usado para exibir feitos...,31/08/2022 12:21,2,0,críticas Bolsonaro presidente ameaça Brasil
4,bolsonaro,Bolsonaro cometeu crime contra a humanidade na...,O júri simbólico do Tribunal Permanente dos Po...,01/09/2022 14:08,2,0,condenou crime sentença Bolsonaro


___
## Classificador automático

Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

Vamos analisar o desempenho do governo Bolsonaro entre os anos de 2019 e 2022, com base nas notícias do site IstoÉ Dinheiro. Foram analisadas mais de 800 notícias.

# Criando O Classificador

In [59]:
import re 


def limpador(text):
    punctuation = '[!-.:?;%()‘’“”/]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = text_subbed.lower()
    return text_subbed

In [60]:
train['P_Chave']=train['P_Chave'].astype(str)
train['Titulo']=train['Titulo'].astype(str)
train['Descrição']=train['Descrição'].astype(str)

train['Descrição']=train['Descrição'].apply(limpador)
train['Titulo']=train['Titulo'].apply(limpador)
train['P_Chave']=train['P_Chave'].apply(limpador)

In [61]:
test['P_Chave']=test['P_Chave'].astype(str)
test['Titulo']=test['Titulo'].astype(str)
test['Descrição']=test['Descrição'].astype(str)

test['Descrição']=test['Descrição'].apply(limpador)
test['Titulo']=test['Titulo'].apply(limpador)
test['P_Chave']=test['P_Chave'].apply(limpador)

In [62]:
train0_tit=train.loc[train['Target']==0,['Descrição','Target']]
train1_tit=train.loc[train['Target']==1,['Descrição','Target']]
train2_tit=train.loc[train['Target']==2,['Descrição','Target']]

In [63]:
excluidos=['[…]','nan','–','de','a','o','e','na','no','do','em','ao','da','é','à','os','nas','nos','das','que',
          'dos','as','r','se','para', '', 'como', 'com', 'um', 'nesta', 'diz', 'uma', 'por', 'foi', 'vai', 'após', 'durante',
          'ele', 'ela', 'pelo', 'pela', 'segundafeira', 'terçafeira', 'quartafeira', 'quintafeira', 'sextafeira', 
          'sábado', 'domingo', 'são', 'ser','jair','presidente','bolsonaro']

# Analisando os Títulos (TREINO)

### Lista dos NEGATIVOS (0)

In [64]:
lista_train0_tit=[]
lista_train0_tit_quero=[]
for i in range(0,len(train0_tit)):
    quero=train0_tit.iloc[i,0]
    final = quero.split()
    lista_train0_tit+=(final)
w=0
while w <len(lista_train0_tit):
    if lista_train0_tit[w] not in excluidos:
        lista_train0_tit_quero.append(lista_train0_tit[w])
    w+=1

### Lista dos POSITIVOS (1)

In [65]:
lista_train1_tit=[]
lista_train1_tit_quero=[]
for i in range(0,len(train1_tit)):
    quero=train1_tit.iloc[i,0]
    final = quero.split()
    lista_train1_tit+=(final)
w=0
while w <len(lista_train1_tit):
    if lista_train1_tit[w] not in excluidos:
        lista_train1_tit_quero.append(lista_train1_tit[w])
    w+=1

### Lista dos NEUTROS (2)

In [66]:
lista_train2_tit=[]
lista_train2_tit_quero=[]
for i in range(0,len(train2_tit)):
    quero=train2_tit.iloc[i,0]
    final = quero.split()
    lista_train2_tit+=(final)
w=0
while w <len(lista_train2_tit):
    if lista_train2_tit[w] not in excluidos:
        lista_train2_tit_quero.append(lista_train2_tit[w])
    w+=1

### Transformando para Serie

In [67]:
serie0_tit=pd.Series(lista_train0_tit_quero).value_counts()
serie1_tit=pd.Series(lista_train1_tit_quero).value_counts()
serie2_tit=pd.Series(lista_train2_tit_quero).value_counts()

#print(lista_train1_tit_quero)

## Testando Iterações

In [68]:
lista_total = lista_train2_tit_quero + lista_train1_tit_quero + lista_train0_tit_quero
lista_exclusivas = []

for i in range(0, len(lista_total)):
    palavra = lista_total[i]
    if (palavra not in serie0_tit.index and palavra not in serie1_tit.index) or (palavra not in serie0_tit.index and palavra not in serie2_tit.index) or (palavra not in serie1_tit.index and palavra not in serie2_tit.index):
        lista_exclusivas.append(palavra)

notexclusivo0 = []
for i in range(0, len(serie0_tit)):
    palavra = serie0_tit.index[i]
    if palavra in lista_exclusivas:
        notexclusivo0.append(palavra)

notexclusivo1 = []
for i in range(0, len(serie1_tit)):
    palavra = serie1_tit.index[i]
    if palavra in lista_exclusivas:
        notexclusivo1.append(palavra)

notexclusivo2 = []
for i in range(0, len(serie2_tit)):
    palavra = serie2_tit.index[i]
    if palavra in lista_exclusivas:
        notexclusivo2.append(palavra)


serie0_tit = serie0_tit.drop(notexclusivo0) 
serie1_tit = serie1_tit.drop(notexclusivo1)
serie2_tit = serie2_tit.drop(notexclusivo2)

In [69]:
len0 = len(serie0_tit)
len1 = len(serie1_tit)
len2 = len(serie2_tit)

if len0 < len1 and len0 < len2:
    serie1_tit = serie1_tit.to_frame()
    serie1_tit = serie1_tit.iloc[0:len0,:]
    serie2_tit = serie2_tit.to_frame()
    serie2_tit = serie2_tit.iloc[0:len0,:]
elif len1 < len2:
    serie0_tit = serie0_tit.to_frame()
    serie0_tit = serie0_tit.iloc[0:len1,:]
    serie2_tit = serie2_tit.to_frame()
    serie2_tit = serie2_tit.iloc[0:len0,:]
else:
    serie0_tit = serie0_tit.to_frame()
    serie0_tit = serie0_tit.iloc[0:len2,:]
    serie1_tit = serie1_tit.to_frame()
    serie1_tit = serie1_tit.iloc[0:len0,:]

serie0_tit = serie0_tit.squeeze()
serie1_tit = serie1_tit.squeeze()
serie2_tit = serie2_tit.squeeze()

In [70]:
for palavra in lista_train0_tit_quero:
    if palavra not in serie0_tit.index:
        lista_train0_tit_quero.remove(palavra)

for palavra in lista_train1_tit_quero:
    if palavra not in serie1_tit.index:
        lista_train1_tit_quero.remove(palavra)
    
for palavra in lista_train2_tit_quero:
    if palavra not in serie2_tit.index:
        lista_train2_tit_quero.remove(palavra)

# Analisando Probabilidades

In [71]:
tam0=len(train0_tit)
tam1=len(train1_tit)
tam2=len(train2_tit)
total= tam0 + tam1 + tam2

prob0=tam0/total
prob1=tam1/total
prob2=tam2/total

print(f'Probabilidade NEGATIVAS/TOTAL: {float(prob0):.5f}')
print(f'Probabilidade POSITIVAS/TOTAL: {float(prob1):.5f}')
print(f'Probabilidade NEUTRAS/TOTAL: {float(prob2):.5f}')

Probabilidade NEGATIVAS/TOTAL: 0.21875
Probabilidade POSITIVAS/TOTAL: 0.26667
Probabilidade NEUTRAS/TOTAL: 0.51458


# Analisando o Desempenho do Classificador

In [72]:
test['Auto_Target']='Vazio'
test_tit=test.loc[:,['Titulo','Target','Auto_Target']]

In [73]:
test_tit['Target'].value_counts()

2    187
1     80
0     53
Name: Target, dtype: int64

In [74]:
neutro = positivo = negativo = 0
contagem_0 = len(lista_train0_tit_quero)
contagem_1 = len(lista_train1_tit_quero)
contagem_2 = len(lista_train2_tit_quero)

lista_total = lista_train2_tit_quero + lista_train1_tit_quero + lista_train0_tit_quero
conta_palavras = []
for i in range(0, len(lista_total)):
    palavra = lista_total[i]
    if palavra not in conta_palavras:
        conta_palavras.append(palavra)
universo = len(conta_palavras)


for i in range(0, len(test_tit)):

    frases = ''
    lista_splits = []
    probFraseDado0 = 0
    probFraseDado1 = 0
    probFraseDado2 = 0

    frases += test_tit.iloc[i,0]
    lista_splits = frases.split()
    '''conta_palavras_test = []
    for w in range(0, len(lista_splits)):
        palavra = lista_splits[w]
        if palavra not in conta_palavras:
            conta_palavras.append(palavra)
    #universo += len(conta_palavras_test)'''

    for w in range(0, len(lista_splits)):
        palavra = lista_splits[w]
        if palavra in serie0_tit.index.to_list():
            if probFraseDado0 == 0:
                probFraseDado0 = (serie0_tit[palavra]+1) / (contagem_0 + universo)
            else:
                probFraseDado0 *= (serie0_tit[palavra]+1) / (contagem_0 + universo)
        else:
            if probFraseDado0 == 0:
                probFraseDado0 = 1 / (contagem_0 + universo)
            else:
                probFraseDado0 *= 1 / (contagem_0 + universo)
        if palavra in serie1_tit.index.to_list():
            if probFraseDado1 == 0:
                probFraseDado1 = (serie1_tit[palavra]+1) / (contagem_1 + universo)
            else:
                probFraseDado1 *= (serie1_tit[palavra]+1) / (contagem_1 + universo)
        else:
            if probFraseDado1 == 0:
                probFraseDado1 = 1 / (contagem_1 + universo)
            else:
                probFraseDado1 *= 1 / (contagem_1 + universo)
        if palavra in serie2_tit.index.to_list():
            if probFraseDado2 == 0:
                probFraseDado2 *= (serie2_tit[palavra]+1) / (contagem_2 + universo)
            else:
                probFraseDado2 *= (serie2_tit[palavra]+1) / (contagem_2 + universo)
        else:
            if probFraseDado2 == 0:
                probFraseDado2 = 1 / (contagem_2 + universo)
            else:
                probFraseDado2 *= 1 / (contagem_2 + universo)

    prob0DadoFrase = probFraseDado0 * prob0
    prob1DadoFrase = probFraseDado1 * prob1
    prob2DadoFrase = probFraseDado2 * prob2

    if float(prob0DadoFrase) > float(prob1DadoFrase) and float(prob0DadoFrase) > float(prob2DadoFrase):
        negativo += 1
        test_tit.iloc[i, 2] = 0

    elif float(prob1DadoFrase) > float(prob2DadoFrase):
        positivo += 1
        test_tit.iloc[i, 2] = 1

    else:
        neutro += 1
        test_tit.iloc[i, 2] = 2


In [75]:
print(f'Bot NEGATIVO: {negativo}')
print(f'Bot POSITIVO: {positivo}')
print(f'Bot NEUTRO: {neutro}')

Bot NEGATIVO: 140
Bot POSITIVO: 70
Bot NEUTRO: 110


In [76]:
falso0 = 0
falso1 = 0
falso2 = 0

for i in range(0, len(test_tit)):
    if test_tit.iloc[i, 1] != test_tit.iloc[i, 2]:
        if test_tit.iloc[i, 2] == 0:
            falso0 += 1
        elif test_tit.iloc[i, 2] == 1:
            falso1 += 1
        else:
            falso2 += 1
        
print(f'Bot FAKE NEGATIVO: {falso0}')
print(f'Bot FAKE POSITIVO: {falso1}')
print(f'Bot FAKE NEUTRO: {falso2}')

Bot FAKE NEGATIVO: 113
Bot FAKE POSITIVO: 42
Bot FAKE NEUTRO: 35


In [77]:
acuracia = 1- (falso0 + falso1 + falso2) / len(test)

print(acuracia)

0.40625


In [78]:
test_tit.head(10)

,Titulo,Target,Auto_Target
0,bolsonaro critica tse e diz que medidas da cor...,0,0
1,esquerda se solidarizou com facada em 2018 ao ...,0,2
2,bolsonaro cometeu crime contra a humanidade na...,0,0
3,crítico de bolsonaro compra domínio e recria s...,0,0
4,bolsonaro cometeu crime contra a humanidade na...,0,0
5,pgr defende prorrogar investigação de bolsonar...,0,2
6,consultorias de orçamento discordam de veto à ...,0,0
7,entrevista de bolsonaro a jn tem panelaços em ...,0,2
8,lula tem 47 contra 32 de bolsonaro diz datafolha,2,2
9,bolsonaro passou ao centrão área com licitação...,0,0


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**